In [1]:
import pandas as pd
import warnings
import numpy as np
import numpy.typing as npt
import aes
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore", category=SyntaxWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

io_data = pd.read_csv('data/dpa-aes/aes_data.txt', header=None)
io_data = io_data.apply(lambda x: x.str.replace(' ', '')) # https://stackoverflow.com/questions/40083266/replace-comma-with-dot-pandas
io_data['in'] = io_data[0].str[:32]
io_data['out'] = io_data[0].str[32:]
io_data = io_data.drop(columns=[0])

# https://stackoverflow.com/questions/40083266/replace-comma-with-dot-pandas
traces = pd.read_csv('data/dpa-aes/aes_samples.txt', sep='\s+', decimal=',', header=None)
# print(io_data['out'].iloc[0])

In [3]:
def hamming_weight(n):
    count = 0
    while n:
        count += n & 1
        n >>= 1
    return count

hamming_weights = np.array([hamming_weight(i) for i in range(256)])

print(hamming_weights[233])

5


In [4]:
plots = []
maxes = np.zeros((2**8, 16), dtype=np.float64)
all_hamming_weights_per_box = np.zeros((5000, 2**8, 16), dtype=np.uint8)
coeff = np.zeros((16, 2**8))

for key_guess in range(2**8):
    print(f'Analyzing key byte guess: {hex(key_guess)}')
    key_guess_matrix = np.full((4, 4), key_guess, dtype=np.uint8)
    for index, in_data in enumerate(io_data['in']):
        box_index = 0
        # fancy way to parse hex data and put it into a 4x4 matrix
        tmp = [int(in_data[i:i + 2], 16) for i in range(0, len(in_data), 2)]
        in_matrix = np.asarray(tmp, dtype=np.uint8).reshape(4, 4)

        added_key = aes.add_round_key(in_matrix, key_guess_matrix)
        sub_out = aes.sub_bytes(added_key, False)

        for row in range(sub_out.shape[0]):
            for col in range(sub_out.shape[1]):
                all_hamming_weights_per_box[index][key_guess][box_index] = hamming_weights[sub_out[row, col]]
                box_index += 1

    for box in range(16): # TODO: should be a way to make this faster
        guess_box_weights = all_hamming_weights_per_box[:, key_guess, box]
        WxH = (traces.T * guess_box_weights).T
        EWxH = np.sum(WxH, axis=0)

        Ew = np.sum(traces, axis=0)
        Eh = np.sum(guess_box_weights)

        Ew2 = np.sum(np.square(traces), axis=0)
        Ew_squared = np.square(Ew)

        Eh2 = np.sum(np.square(guess_box_weights), axis=0)
        Eh_squared = np.square(Eh)

        top = (5000 * EWxH) - (Eh * Ew)
        bottom = np.sqrt((5000 * Ew2) - Ew_squared) * np.sqrt((5000 * Eh2) - Eh_squared)
        coeff[box][key_guess] = np.max(top / bottom)


Analyzing key byte guess: 0x0
Analyzing key byte guess: 0x1
Analyzing key byte guess: 0x2
Analyzing key byte guess: 0x3
Analyzing key byte guess: 0x4
Analyzing key byte guess: 0x5
Analyzing key byte guess: 0x6
Analyzing key byte guess: 0x7
Analyzing key byte guess: 0x8
Analyzing key byte guess: 0x9
Analyzing key byte guess: 0xa
Analyzing key byte guess: 0xb
Analyzing key byte guess: 0xc
Analyzing key byte guess: 0xd
Analyzing key byte guess: 0xe
Analyzing key byte guess: 0xf
Analyzing key byte guess: 0x10
Analyzing key byte guess: 0x11
Analyzing key byte guess: 0x12
Analyzing key byte guess: 0x13
Analyzing key byte guess: 0x14
Analyzing key byte guess: 0x15
Analyzing key byte guess: 0x16
Analyzing key byte guess: 0x17
Analyzing key byte guess: 0x18
Analyzing key byte guess: 0x19
Analyzing key byte guess: 0x1a
Analyzing key byte guess: 0x1b
Analyzing key byte guess: 0x1c
Analyzing key byte guess: 0x1d
Analyzing key byte guess: 0x1e
Analyzing key byte guess: 0x1f
Analyzing key byte guess

In [ ]:
# This can be done inside the main loop...
# for box in range(16):
#     for guess in range(2**8):
#         guess_box_weights = all_hamming_weights_per_box[:, guess, box]
#         WxH = (traces.T * guess_box_weights).T
#         EWxH = np.sum(WxH, axis=0)

#         Ew = np.sum(traces, axis=0)
#         Eh = np.sum(guess_box_weights)

#         Ew2 = np.sum(np.square(traces), axis=0)
#         Ew_squared = np.square(Ew)

#         Eh2 = np.sum(np.square(guess_box_weights), axis=0)
#         Eh_squared = np.square(Eh)

#         top = (5000 * EWxH) - (Eh * Ew)
#         bottom = np.sqrt((5000 * Ew2) - Ew_squared) * np.sqrt((5000 * Eh2) - Eh_squared)
#         coeff[box][guess] = np.max(top / bottom)

In [5]:
for box in range(16):
    print(hex(np.argsort(coeff[box, :])[-1]))

0xca
0xfe
0xba
0xbe
0xca
0xfe
0xba
0xbe
0xca
0xfe
0xba
0xbe
0xca
0xfe
0xba
0xbe
